In [52]:
import pandas as pd
import json
from pandas import json_normalize

In [53]:
df = pd.read_json('data/russia-cities.json')

cities_df = df.sort_values('population', ascending=False).head(10)

cities_df

,name,name_alt,label,type,typeShort,contentType,id,okato,oktmo,guid,...,isCapital,zip,population,yearFounded,yearCityStatus,name_en,namecase,coords,timezone,region
589,Москва,Москва,moscow,Город,г,city,7700000000000,45000000000,45000000,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,...,True,111674,13010112,1147,1147,Moscow,"{'nominative': 'Москва', 'genitive': 'Москвы',...","{'lat': 55.7540471, 'lon': 37.620405}","{'tzid': 'Europe/Moscow', 'abbreviation': 'MSK...","{'name': 'Москва', 'label': 'moscow', 'type': ..."
570,Санкт-Петербург,Санкт-Петербург,saint_petersburg,Город,г,city,7800000000000,40000000000,40000000,c2deb16a-0330-4f05-821f-1d09c93331e6,...,True,190000,5601911,1703,1703,Saint Petersburg,"{'nominative': 'Санкт-Петербург', 'genitive': ...","{'lat': 59.943646, 'lon': 30.2851863}","{'tzid': 'Europe/Moscow', 'abbreviation': 'MSK...","{'name': 'Санкт-Петербург', 'label': 'saint_pe..."
718,Новосибирск,Новосибирск,novosibirsk,Город,г,city,5400000100000,50401000000,50701000001,8dea00e3-9aab-4d8e-887c-ef2aaa546456,...,True,630132,1633595,1893,1903,Novosibirsk,"{'nominative': 'Новосибирск', 'genitive': 'Нов...","{'lat': 55.028191, 'lon': 82.9211489}","{'tzid': 'Asia/Krasnoyarsk', 'abbreviation': '...","{'name': 'Новосибирская', 'label': 'novosibirs..."
885,Екатеринбург,Екатеринбург,yekaterinburg,Город,г,city,6600000100000,65401000000,65701000001,2763c110-cb8b-416a-9dac-ad28a55b4402,...,True,620043,1544376,1723,1781,Yekaterinburg,"{'nominative': 'Екатеринбург', 'genitive': 'Ек...","{'lat': 56.8385216, 'lon': 60.6054911}","{'tzid': 'Asia/Yekaterinburg', 'abbreviation':...","{'name': 'Свердловская', 'label': 'sverdlovska..."
125,Казань,Казань,kazan,Город,г,city,1600000100000,92401000000,92701000001,93b3df57-4c89-44df-ac42-96f05e9cd3b9,...,True,420032,1308660,1005,1005,Kazan,"{'nominative': 'Казань', 'genitive': 'Казани',...","{'lat': 55.7943584, 'lon': 49.1114975}","{'tzid': 'Europe/Moscow', 'abbreviation': 'MSK...","{'name': 'Татарстан', 'label': 'tatarstan', 't..."
700,Нижний Новгород,Нижний Новгород,nizhny_novgorod,Город,г,city,5200000100000,22401000000,22701000001,555e7d61-d9a7-4ba6-9770-6caa8198c483,...,True,603069,1226076,1221,1221,Nizhny Novgorod,"{'nominative': 'Нижний Новгород', 'genitive': ...","{'lat': 56.3240627, 'lon': 44.0053913}","{'tzid': 'Europe/Moscow', 'abbreviation': 'MSK...","{'name': 'Нижегородская', 'label': 'nizhegorod..."
1041,Челябинск,Челябинск,chelyabinsk,Город,г,city,7400000100000,75401000000,75701000001,a376e68d-724a-4472-be7c-891bdb09ae32,...,True,454004,1189525,1736,1781,Chelyabinsk,"{'nominative': 'Челябинск', 'genitive': 'Челяб...","{'lat': 55.1602624, 'lon': 61.4008078}","{'tzid': 'Asia/Yekaterinburg', 'abbreviation':...","{'name': 'Челябинская', 'label': 'chelyabinska..."
214,Красноярск,Красноярск,krasnoyarsk,Город,г,city,2400000100000,4401000000,4701000001,9b968c73-f4d4-4012-8da8-3dacd4d4c1bd,...,True,660001,1187771,1628,1690,Krasnoyarsk,"{'nominative': 'Красноярск', 'genitive': 'Крас...","{'lat': 56.0093879, 'lon': 92.8524806}","{'tzid': 'Asia/Krasnoyarsk', 'abbreviation': '...","{'name': 'Красноярский', 'label': 'krasnoyarsk..."
832,Самара,Самара,samara,Город,г,city,6300000100000,36401000000,36701000,bb035cc3-1dc2-4627-9d25-a1bf2d4b936b,...,True,443016,1173299,1586,1688,Samara,"{'nominative': 'Самара', 'genitive': 'Самары',...","{'lat': 53.1950306, 'lon': 50.1069518}","{'tzid': 'Europe/Samara', 'abbreviation': 'SAM...","{'name': 'Самарская', 'label': 'samarskaya', '..."
10,Уфа,Уфа,ufa,Город,г,city,200000100000,80401000000,80701000,7339e834-2cb4-4734-a4c7-1fca2c66e562,...,True,450019,1144809,1574,1586,Ufa,"{'nominative': 'Уфа', 'genitive': 'Уфы', 'dati...","{'lat': 54.734944, 'lon': 55.9578468}","{'tzid': 'Asia/Yekaterinburg', 'abbreviation':...","{'name': 'Башкортостан', 'label': 'bashkortost..."


Из http://dx.doi.org/10.1134/S2079970517010051

Средний поток МСК --- МО = 0.85

Расстояние 50 км

https://t-j.ru/kuda-letim-avia-stat/

In [56]:
from geopy.distance import geodesic
import numpy as np

flows = np.zeros((len(cities_df['label']),len(cities_df['label'])))

for i, label_i in enumerate(cities_df['label']):
    for j, label_j in enumerate(cities_df['label']):
        if i!=j:
            dist = geodesic((cities_df['coords'].values[i]['lat'],cities_df['coords'].values[i]['lon']), 
                                           (cities_df['coords'].values[j]['lat'],cities_df['coords'].values[j]['lon'])).kilometers
            
            flows[i,j] = cities_df['population'].values[i] * cities_df['population'].values[j] / dist**1.5
            # print(label_i, label_j, flows[i,j])

flows = flows*8e7 / np.sum(flows)
flows = flows / 365

for i, label_i in enumerate(cities_df['label']):
    for j, label_j in enumerate(cities_df['label']):
        if i!=j:
            flows[i,j] /= cities_df['population'].values[i]

np.save('data/flows.npy', flows)
flows

array([[0.00000000e+00, 2.89913319e-03, 9.08778564e-05, 2.40113189e-04,
        5.64027771e-04, 1.26346596e-03, 1.70863441e-04, 5.07455301e-05,
        3.89884227e-04, 2.38846456e-04],
       [6.73306797e-03, 0.00000000e+00, 7.82131066e-05, 1.69910564e-04,
        2.61060603e-04, 3.77776976e-04, 1.18049329e-04, 4.60550358e-05,
        1.82078171e-04, 1.43829760e-04],
       [7.23760228e-04, 2.68207764e-04, 0.00000000e+00, 2.44925347e-04,
        1.11566599e-04, 8.58244709e-05, 1.95765380e-04, 6.16776499e-04,
        9.90453583e-05, 1.33707874e-04],
       [2.02275837e-03, 6.16316141e-04, 2.59074747e-04, 0.00000000e+00,
        5.64618380e-04, 3.13963510e-04, 3.68703158e-03, 1.12880535e-04,
        4.45930255e-04, 1.31961550e-03],
       [5.60731165e-03, 1.11750818e-03, 1.39268135e-04, 6.66317512e-04,
        0.00000000e+00, 1.75627986e-03, 4.55646527e-04, 7.09376998e-05,
        1.91547664e-03, 9.96857101e-04],
       [1.34068635e-02, 1.72605368e-03, 1.14350519e-04, 3.95471170e-04,
   

In [61]:
pop_sizes = cities_df['population'].values
pop_sizes = np.floor(pop_sizes * 1e6 / np.sum(pop_sizes))

np.save('data/sizes.npy', pop_sizes)
pop_sizes

array([448313., 193035.,  56291.,  53217.,  45094.,  42249.,  40989.,
        40929.,  40430.,  39448.])

In [62]:
labels = cities_df['label'].values

np.save('data/labels.npy', labels)
labels

array(['moscow', 'saint_petersburg', 'novosibirsk', 'yekaterinburg',
       'kazan', 'nizhny_novgorod', 'chelyabinsk', 'krasnoyarsk', 'samara',
       'ufa'], dtype=object)